In [ ]:
#Setup packages and sqlite connection
import sqlite3 as sq
import pandas as pd
import numpy as np
cnx = sq.connect('test.db')
cur = cnx.cursor()

In [ ]:
#Load Data from PACS1
#Set PK from PACS1 (PAT_ID) as Index
dfpatientextract = pd.read_csv('PACS1patientextract05012019.csv' 
                               ,header=0
                               ,converters={'PACS1_MRN':str, 'PACS1_MRN_9_Digits':str, 'PACS1_MRN_12_Digits':str}
                               ,index_col = 'PACS1_PAT_ID'
                               ,parse_dates=['PACS1_DOB'], date_parser = lambda d: pd.to_datetime(d, format = '%Y%m%d', errors='coerce')
                               )
dfpatientextract.shape

In [ ]:
dfpatientextract.head()

In [ ]:
#Load Crosswalk data from EDW team, sheet by sheet
dfcurrentmrnmatch = pd.read_excel('PACS1_Mapping.xlsx'
                                  ,sheet_name = 'Current MRN mapping'
                                  ,header=0
                                  ,index_col = 'PAT_ID'
                                  ,usecols = ['PAT_ID', 'Clarity_MRN','Clarity_PAT_NAME','Clarity_DOB','Clarity_SEX']
                                #  ,names = ['Mapped_MRN','Clarity_PAT_NAME','Clarity_DOB','Mapped_Gender']
                                  ,converters={'Clarity_MRN':str}
                                # ,parse_dates=['Clarity_DOB'], date_parser = lambda d: pd.Series.dt.strftime(d, format="%Y%m%d")
                                 )

dfcurrentmrnmatch.shape

In [ ]:
dfhistoricalmrnmatch = pd.read_excel('PACS1_Mapping.xlsx'
                                  ,sheet_name = 'Historical MRN Mapping'
                                  ,header=0
                                  ,index_col = 'PAT_ID'
                                  ,usecols = ['PAT_ID', 'Clarity_New_MRN_2','Clarity_PAT_NAME','CLARITY_DOB','Clarity_SEX']
                                 # ,names = ['PAT_ID','Mapped_MRN','Clarity_PAT_NAME','Clarity_DOB','Mapped_Gender']
                                  ,converters={'Clarity_New_MRN_2':str}
                                 ,parse_dates=['Clarity_DOB'], date_parser = lambda d: pd.Series.dt.strftime(d, format="%Y%m%d")
                                 )

dfhistoricalmrnmatch.shape

In [ ]:
dfnomrnmatch = pd.read_excel('RemainingToBeMapped.xlsx'
                                  ,sheet_name = 'No Mapping'
                                  ,header=0
                                  ,index_col = 'PAT_ID'
                                  ,usecols = ['PAT_ID', 'PACS1_MRN','NAME_L','NAME_F','DOB','GENDER']
                                 # ,names = ['PAT_ID','Mapped_MRN','Clarity_PAT_NAME','Clarity_DOB','Mapped_Gender']
                                  ,converters={'PACS1_MRN':str}
                                 ,parse_dates=['Clarity_DOB'], date_parser = lambda d: pd.Series.dt.strftime(d, format="%Y%m%d")
                            )
dfnomrnmatch.shape

In [ ]:
dfnewmrnmatch = pd.read_excel('RemainingToBeMapped.xlsx'
                                 ,sheet_name = '5.20.2019 Mapping'
                                 ,header = 0
                                 ,index_col = 'PAT_ID'
                                 ,usecols = ['PAT_ID','Clarity_MRN','Clarity_new_MRN','Clarity_PAT_NAME','Clarity_DOB','Clarity_SEX']
                                 ,converters = {'Clarity_MRN':str, 'Clarity_new_MRN':str}
                             )
dfnewmrnmatch.shape

In [ ]:
# cur.execute('''CREATE TABLE currentmrnmatch (
#                 PAT_ID varchar(10) NOT NULL PRIMARY KEY 
#                 ,Clarity_MRN varchar(12)
#                 ,Clarity_PAT_NAME varchar(112)
#                 ,Clarity_DOB datetime
#                 ,Clarity_SEX varchar(2)
#                 )
#                 ''')

# cur.execute('''CREATE TABLE historicalmrnmatch (
#                 PAT_ID varchar(10) NOT NULL PRIMARY KEY 
#                 ,Clarity_New_MRN_2 varchar(12)
#                 ,Clarity_PAT_NAME varchar(112)
#                 ,CLARITY_DOB datetime
#                 ,Clarity_SEX varchar(2)
#                 )
#                 ''')

In [ ]:
# dfcurrentmrnmatch.to_sql("currentmrnmatch", cnx, if_exists='replace', index='PAT_ID')
# dfhistoricalmrnmatch.to_sql("historicalmrnmatch", cnx, if_exists='replace', index='PAT_ID')
# dfnomrnmatch.to_sql("nomrnmatch", cnx, if_exists='replace', index='PAT_ID')
# dfnewmrnmatch.to_sql("newmrnmatch", cnx, if_exists='replace', index='PAT_ID')

In [ ]:
dfcurrentmrnmatch =pd.read_sql_query("""SELECT * FROM currentmrnmatch;""", cnx, index_col='PAT_ID')
dfhistoricalmrnmatch =pd.read_sql_query("""SELECT * FROM historicalmrnmatch;""", cnx, index_col='PAT_ID')
dfnomrnmatch =pd.read_sql_query("""SELECT * FROM nomrnmatch;""", cnx, index_col='PAT_ID')
dfnewmrnmatch =pd.read_sql_query("""SELECT * FROM newmrnmatch;""", cnx, index_col='PAT_ID')

In [ ]:
dfpatientextract.info()

In [ ]:
dfnewmrnmatch.loc[~dfnewmrnmatch['Clarity_MRN'].isnull(),'Mapped_MRN'] = dfnewmrnmatch['Clarity_MRN'][~dfnewmrnmatch['Clarity_MRN'].isnull()]
dfnewmrnmatch.loc[dfnewmrnmatch['Clarity_MRN'].isnull(),'Mapped_MRN'] = dfnewmrnmatch['Clarity_new_MRN'][dfnewmrnmatch['Clarity_MRN'].isnull()]
dfnewmrnmatch.drop(columns=['Clarity_MRN','Clarity_new_MRN'], inplace=True)

In [ ]:
dfnomrnmatch.rename(columns = {'PACS1_MRN':'PACS1_MRN_12_Digits'}, inplace=True)
dfcurrentmrnmatch.rename(columns={'Clarity_MRN':'Mapped_MRN','Clarity_DOB':'Mapped_DOB','Clarity_SEX':'Mapped_Gender'},inplace=True)
dfhistoricalmrnmatch.rename(columns={'Clarity_New_MRN_2':'Mapped_MRN','CLARITY_DOB':'Mapped_DOB','Clarity_SEX':'Mapped_Gender'}, inplace=True)
dfnewmrnmatch.rename(columns ={'Clarity_DOB':'Mapped_DOB','Clarity_SEX':'Mapped_Gender'}, inplace=True )
dfpremapping = dfcurrentmrnmatch.append(dfhistoricalmrnmatch, sort=False)
dfpremapping = dfpremapping.append(dfnewmrnmatch, sort=False)
dfpremapping.info()

In [ ]:
dfpremapping[['Mapped_Last_Name','Mapped_First_Names']] = dfpremapping.Clarity_PAT_NAME.str.split(",",expand=True)
dfpremapping['Mapped_First_Name'] = dfpremapping.Mapped_First_Names.str.split(expand=True)[0]
dfmapping = dfpatientextract.join(dfpremapping, how='left')
dfmapping.shape

In [ ]:
dfhasmapping = dfmapping[['PACS1_MRN', 'Mapped_MRN',
                         'PACS1_Last_Name','Mapped_Last_Name',
                         'PACS1_First_Name','Mapped_First_Name',
                         'PACS1_Gender','Mapped_Gender',
                         'PACS1_DOB','Mapped_DOB']][~dfmapping['Mapped_MRN'].isnull()].copy()
dfhasmapping['Mapped_DOB'] = pd.to_datetime(dfhasmapping['Mapped_DOB'])
dfhasmapping.shape

In [ ]:
dfhasmapping['Conflicts'] = 0
dfhasmapping['Conflict_Type'] = ''

In [ ]:
dfhasmapping.columns

In [ ]:
dfhasmapping.info()

In [ ]:
#Set conflict count and conflict type type for each possible conflict type. Possibly could create a function or something that's
#not so verbose

dfhasmapping.loc[(dfhasmapping['PACS1_Last_Name'] == dfhasmapping['Mapped_Last_Name']) &
                 (dfhasmapping['PACS1_First_Name'] == dfhasmapping['Mapped_First_Name']) &
                 (dfhasmapping['PACS1_Gender'] == dfhasmapping['Mapped_Gender']) &
                 (dfhasmapping['PACS1_DOB'] == dfhasmapping['Mapped_DOB']),['Conflicts','Conflict_Type']] = 0,'MATCH'

dfhasmapping.loc[(dfhasmapping['PACS1_Last_Name'] != dfhasmapping['Mapped_Last_Name']) &
                 (dfhasmapping['PACS1_First_Name'] == dfhasmapping['Mapped_First_Name']) &
                 (dfhasmapping['PACS1_Gender'] == dfhasmapping['Mapped_Gender']) &
                 (dfhasmapping['PACS1_DOB'] == dfhasmapping['Mapped_DOB']),['Conflicts','Conflict_Type']] = 1,'L'

dfhasmapping.loc[(dfhasmapping['PACS1_Last_Name'] == dfhasmapping['Mapped_Last_Name']) &
                 (dfhasmapping['PACS1_First_Name'] != dfhasmapping['Mapped_First_Name']) &
                 (dfhasmapping['PACS1_Gender'] == dfhasmapping['Mapped_Gender']) &
                 (dfhasmapping['PACS1_DOB'] == dfhasmapping['Mapped_DOB']),['Conflicts','Conflict_Type']] = 1,'F'

dfhasmapping.loc[(dfhasmapping['PACS1_Last_Name'] == dfhasmapping['Mapped_Last_Name']) &
                 (dfhasmapping['PACS1_First_Name'] == dfhasmapping['Mapped_First_Name']) &
                 (dfhasmapping['PACS1_Gender'] != dfhasmapping['Mapped_Gender']) &
                 (dfhasmapping['PACS1_DOB'] == dfhasmapping['Mapped_DOB']),['Conflicts','Conflict_Type']] = 1,'G'

dfhasmapping.loc[(dfhasmapping['PACS1_Last_Name'] == dfhasmapping['Mapped_Last_Name']) &
                 (dfhasmapping['PACS1_First_Name'] == dfhasmapping['Mapped_First_Name']) &
                 (dfhasmapping['PACS1_Gender'] == dfhasmapping['Mapped_Gender']) &
                 (dfhasmapping['PACS1_DOB'] != dfhasmapping['Mapped_DOB']),['Conflicts','Conflict_Type']] = 1,'D'

dfhasmapping.loc[(dfhasmapping['PACS1_Last_Name'] != dfhasmapping['Mapped_Last_Name']) &
                 (dfhasmapping['PACS1_First_Name'] != dfhasmapping['Mapped_First_Name']) &
                 (dfhasmapping['PACS1_Gender'] == dfhasmapping['Mapped_Gender']) &
                 (dfhasmapping['PACS1_DOB'] == dfhasmapping['Mapped_DOB']),['Conflicts','Conflict_Type']] = 2,'LF'

dfhasmapping.loc[(dfhasmapping['PACS1_Last_Name'] != dfhasmapping['Mapped_Last_Name']) &
                 (dfhasmapping['PACS1_First_Name'] == dfhasmapping['Mapped_First_Name']) &
                 (dfhasmapping['PACS1_Gender'] != dfhasmapping['Mapped_Gender']) &
                 (dfhasmapping['PACS1_DOB'] == dfhasmapping['Mapped_DOB']),['Conflicts','Conflict_Type']] = 2,'LG'

dfhasmapping.loc[(dfhasmapping['PACS1_Last_Name'] != dfhasmapping['Mapped_Last_Name']) &
                 (dfhasmapping['PACS1_First_Name'] == dfhasmapping['Mapped_First_Name']) &
                 (dfhasmapping['PACS1_Gender'] == dfhasmapping['Mapped_Gender']) &
                 (dfhasmapping['PACS1_DOB'] != dfhasmapping['Mapped_DOB']),['Conflicts','Conflict_Type']] = 2,'LD'

dfhasmapping.loc[(dfhasmapping['PACS1_Last_Name'] == dfhasmapping['Mapped_Last_Name']) &
                 (dfhasmapping['PACS1_First_Name'] != dfhasmapping['Mapped_First_Name']) &
                 (dfhasmapping['PACS1_Gender'] != dfhasmapping['Mapped_Gender']) &
                 (dfhasmapping['PACS1_DOB'] == dfhasmapping['Mapped_DOB']),['Conflicts','Conflict_Type']] = 2,'FG'

dfhasmapping.loc[(dfhasmapping['PACS1_Last_Name'] == dfhasmapping['Mapped_Last_Name']) &
                 (dfhasmapping['PACS1_First_Name'] != dfhasmapping['Mapped_First_Name']) &
                 (dfhasmapping['PACS1_Gender'] == dfhasmapping['Mapped_Gender']) &
                 (dfhasmapping['PACS1_DOB'] != dfhasmapping['Mapped_DOB']),['Conflicts','Conflict_Type']] = 2,'FD'

dfhasmapping.loc[(dfhasmapping['PACS1_Last_Name'] == dfhasmapping['Mapped_Last_Name']) &
                 (dfhasmapping['PACS1_First_Name'] == dfhasmapping['Mapped_First_Name']) &
                 (dfhasmapping['PACS1_Gender'] != dfhasmapping['Mapped_Gender']) &
                 (dfhasmapping['PACS1_DOB'] != dfhasmapping['Mapped_DOB']),['Conflicts','Conflict_Type']] = 2,'GD'

dfhasmapping.loc[(dfhasmapping['PACS1_Last_Name'] != dfhasmapping['Mapped_Last_Name']) &
                 (dfhasmapping['PACS1_First_Name'] != dfhasmapping['Mapped_First_Name']) &
                 (dfhasmapping['PACS1_Gender'] != dfhasmapping['Mapped_Gender']) &
                 (dfhasmapping['PACS1_DOB'] == dfhasmapping['Mapped_DOB']),['Conflicts','Conflict_Type']] = 3,'LFG'

dfhasmapping.loc[(dfhasmapping['PACS1_Last_Name'] != dfhasmapping['Mapped_Last_Name']) &
                 (dfhasmapping['PACS1_First_Name'] != dfhasmapping['Mapped_First_Name']) &
                 (dfhasmapping['PACS1_Gender'] == dfhasmapping['Mapped_Gender']) &
                 (dfhasmapping['PACS1_DOB'] != dfhasmapping['Mapped_DOB']),['Conflicts','Conflict_Type']] = 3,'LFD'

dfhasmapping.loc[(dfhasmapping['PACS1_Last_Name'] != dfhasmapping['Mapped_Last_Name']) &
                 (dfhasmapping['PACS1_First_Name'] == dfhasmapping['Mapped_First_Name']) &
                 (dfhasmapping['PACS1_Gender'] != dfhasmapping['Mapped_Gender']) &
                 (dfhasmapping['PACS1_DOB'] != dfhasmapping['Mapped_DOB']),['Conflicts','Conflict_Type']] = 3,'LGD'

dfhasmapping.loc[(dfhasmapping['PACS1_Last_Name'] == dfhasmapping['Mapped_Last_Name']) &
                 (dfhasmapping['PACS1_First_Name'] != dfhasmapping['Mapped_First_Name']) &
                 (dfhasmapping['PACS1_Gender'] != dfhasmapping['Mapped_Gender']) &
                 (dfhasmapping['PACS1_DOB'] != dfhasmapping['Mapped_DOB']),['Conflicts','Conflict_Type']] = 3,'FGD'

dfhasmapping.loc[(dfhasmapping['PACS1_Last_Name'] != dfhasmapping['Mapped_Last_Name']) &
                 (dfhasmapping['PACS1_First_Name'] != dfhasmapping['Mapped_First_Name']) &
                 (dfhasmapping['PACS1_Gender'] != dfhasmapping['Mapped_Gender']) &
                 (dfhasmapping['PACS1_DOB'] != dfhasmapping['Mapped_DOB']),['Conflicts','Conflict_Type']] = 4,'LFGD'



In [ ]:
#We wan't to map correct value where missing, but want to call out where it is missing

# dfhasmapping.loc[(dfhasmapping['Conflict_Type'].str.contains('G')) & 
#                                ((dfhasmapping['PACS1_Gender'].isnull()) | (dfhasmapping['PACS1_Gender'] == 'MISSING'))
#                 , ['Conflicts']] += -1

## We replace instances of G with '' where mismatch is due to missing PACS1 Gender
dfhasmapping.loc[(dfhasmapping['Conflict_Type'].str.contains('G')) & 
                               ((dfhasmapping['PACS1_Gender'].isnull()) | (dfhasmapping['PACS1_Gender'] == 'MISSING'))
                ,['Conflict_Type']] =\
    dfhasmapping['Conflict_Type'][(dfhasmapping['Conflict_Type'].str.contains('G')) & 
                               ((dfhasmapping['PACS1_Gender'].isnull()) | (dfhasmapping['PACS1_Gender'] == 'MISSING'))].str.\
                                replace(r'G',r'NG')
    
# ##Finally we set conflict type to Match if the only mismatch was due to no Gender in PACS1 
# ##(i.e. Conflict Type is now '' and conflicts = 0 )
# dfhasmapping.loc[(dfhasmapping['Conflicts'] == 0) & (dfhasmapping['Conflict_Type'] == ''), ['Conflict_Type']] = 'MATCH'

#We DO treat missing DB as mismatch, but want to indicate when the mismatch is due to Missing DOB in PACS1. Replace D with ND
dfhasmapping.loc[(dfhasmapping['Conflict_Type'].str.contains('D')) & 
                               ((dfhasmapping['PACS1_DOB'].isnull()) | (dfhasmapping['PACS1_DOB'] == 'MISSING'))
                ,['Conflict_Type']] =\
    dfhasmapping['Conflict_Type'][(dfhasmapping['Conflict_Type'].str.contains('D')) & 
                               ((dfhasmapping['PACS1_DOB'].isnull()) | (dfhasmapping['PACS1_DOB'] == 'MISSING'))].str.\
                                    replace(r'D',r'ND')

In [ ]:
print('Conflict Counts :\n', dfhasmapping.Conflicts.value_counts(dropna=False))
print('\nConflict Counts by Type : \n', dfhasmapping.Conflict_Type.value_counts(dropna=False))

In [ ]:
#List of Perfect Matches
perfectmatch = dfhasmapping[dfhasmapping['Conflicts']==0]
perfectmatch.to_csv("C:\\Users\\username\\Desktop\\perfectmatchPACS1.csv")
perfectmatch.shape

In [ ]:
#Separate conflicts out to work with a smaller dataframe moving forward
dfconflictsmaster = dfhasmapping[dfhasmapping['Conflicts']!=0]
dfconflictsmaster.shape

In [ ]:
#Split out in to number of conflicts
#We are going to count One Conflict as a Match and Ask Secant to Map the conflicting value to the correct one
oneconflict = dfconflictsmaster[dfconflictsmaster['Conflicts']==1]
oneconflict.to_csv("C:\\Users\\username\\Desktop\\oneconflict.csv")
print('One Conflict shape : ', oneconflict.shape)

twoconflict = dfconflictsmaster[dfconflictsmaster['Conflicts']==2]
twoconflict.to_csv("C:\\Users\\username\\Desktop\\twoconflict.csv")
print('\nTwo Conflict shape : ',twoconflict.shape)

threeconflict = dfconflictsmaster[dfconflictsmaster['Conflicts']==3]
threeconflict.to_csv("C:\\Users\\username\\Desktop\\threeconflict.csv")
print('\nThree Conflict shape : ',threeconflict.shape)

fourconflict = dfconflictsmaster[dfconflictsmaster['Conflicts']==4]
fourconflict.to_csv("C:\\Users\\username\\Desktop\\fourconflict.csv")
print('\nFour Conflict shape : ',fourconflict.shape)

In [ ]:
dfnomapping = dfnomrnmatch.join(dfpatientextract['PACS1_MRN'], how='left')
dfnomapping.to_csv("C:\\Users\\username\\Desktop\\nomapping.csv")